In [1]:
from utilities import *
from core.initial import Insertions
from docplex.cp.model import *
from IPython.display import display


In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"

d = DataProvider(filepath=datafile, route='910')

model = CSPModel(d)
model.build_model()

model.data.head()

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
0,Fix Station,Fix Station,NaN,05:00,40,300,340
1,Fix Station,Fix Station,NaN,05:20,40,320,360
2,Fix Station,Fix Station,NaN,05:55,40,355,395
3,Fix Station,Fix Station,NaN,06:20,44,380,424
4,Fix Station,Fix Station,NaN,06:40,44,400,444


In [ ]:
initial = Insertions(model)
initial.solve()

In [3]:
trips = [interval_var(start=(trip.start_time, trip.start_time),
                      end=(trip.end_time, trip.end_time),
                      size=trip.duration,
                      name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [ ]:
ntrips = len(model.trips)
nduties = len(model.trips)

sub = CpoModel(name="Pricing_Subproblem")

# Variables

min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(nduties)]

trip2trip = integer_var_list(size=ntrips,
                             min=0,
                             max=ntrips + 1,
                             name='Trip2Trip')

trip2duty = integer_var_list(size=ntrips,
                             min=0,
                             max=nduties,
                             name='Trip2Duty')

start_times = [[integer_var(min=0,
                            max=model.constraints.shift_span,
                            name=f"StartTime-{i}-{j}")
               for j in range(nduties)] for i in range(ntrips)]

cdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.continuous_driving,
                       name="CDT")

tdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.total_driving,
                       name="TDT")

# Constraints

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_times[i] <= model.start_times[j]))

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_locs[i] <= model.start_locs[j]))

for i in range(ntrips):
    sub.add(trip2trip[i] != i)

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, trip2duty[i] == trip2duty[j]))


In [ ]:
def report_solution(cpsol: CpoSolveResult):
    trips_per_duty = {}
    for i in range(ntrips):
        _out = f"{i:>2} -> {cpsol[trip2trip[i]]} | Duty: {cpsol[trip2duty[i]]}"
        # print(_out)

        duty_id = cpsol[trip2duty[i]]

        if duty_id in trips_per_duty:
            trips_per_duty[duty_id].append(i)
        else:
            trips_per_duty[duty_id] = []
            trips_per_duty[duty_id].append(i)

    print(f'\n\nTotal Duties: {len(trips_per_duty.keys())}')

    for duty_id, duty_trips in trips_per_duty.items():
        df_trips = model.data.loc[duty_trips]

        span = df_trips[end_time].max() - df_trips[start_time].min()

        print(
            f'\n\n>>> Duty {duty_id} - Trips: {len(duty_trips)} - Drive Time: {df_trips[trip_duration].sum()} - Shift Span: {span}\n')
        display(df_trips)

In [4]:
model.trips = model.trips[:10]

In [5]:
ntrips = len(model.trips)
nduties = 3

sub = CpoModel(name="Pricing_Subproblem")

# Variables

min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

trips = [interval_var(start=(trip.start_time, trip.start_time),
                      end=(trip.end_time, trip.end_time),
                      size=trip.duration,
                      name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(nduties)]

breaks = [interval_var(size=model.constraints.break_time,
                       name=f"BreakTime_{i}",
                       optional=True)
          for i in range(nduties)]

In [6]:
trip2duty = {}
for t in range(ntrips):
    for d in range(nduties):
        trip2duty[(t, d)] = sub.interval_var(size=model.durations[t],
                                             name=f"Trip {t}-Duty {d}",
                                             optional=True)


In [7]:
for d in range(nduties):
    sub.add(sub.span(duties[d], [trip2duty[(t,d)] for t in range(ntrips)]))

In [ ]:
for d in range(nduties):
    sub.add(sub.no_overlap([trip2duty[(t,d)] for t in range(ntrips)]))

In [ ]:
for t1 in range(ntrips-1):
    for t2 in range(t1 + 1, ntrips):
        sub.add(end_before_start(trips[t1], trips[t2]))

In [ ]:
cdt = {}
tdt = {}

for d in range(nduties):
    cdt[d] = sub.step_at(0, 0)
    tdt[d] = sub.step_at(0, 0)
    for t in range(ntrips):
        cdt[d] += sub.pulse(trip2duty[(t,d)], model.durations[t])
        tdt[d] += sub.pulse(trip2duty[(t,d)], model.durations[t])

    sub.add(cdt[d] <= model.constraints.continuous_driving)
    sub.add(tdt[d] <= model.constraints.total_driving)
  

In [ ]:
cdt

In [ ]:
obj = sub.sum([sub.presence_of(duty) for duty in duties])
sub.add(sub.minimize(obj))

In [8]:
msol = sub.solve()

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Satisfiability problem - 33 variables, 3 constraints
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 148.6 (before), 148.6 (after)
 !  . Memory usage      : 371.3 kB (before), 371.3 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 ! Using iterative diving.
 *                     28  0.02s        1            -
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 114
 ! Number of fails        : 1
 ! Total memory usage     : 2.0 MB (2.0 MB CP Optimizer + 0.1 MB Concert)
 ! Time spent in solve    : 0.03s (0.02s engine + 0.01s extraction)
 ! Se

AttributeError: 'CpoModel' object has no attribute 'print_solutions'

In [9]:
for i in msol.get_all_var_solutions():
    print(i)

Trip 0-Duty 0: absent
Trip 1-Duty 0: absent
Trip 2-Duty 0: absent
Trip 3-Duty 0: absent
Trip 4-Duty 0: absent
Trip 5-Duty 0: absent
Trip 6-Duty 0: absent
Trip 7-Duty 0: absent
Trip 8-Duty 0: absent
Trip 9-Duty 0: absent
Duty_0: absent
Trip 0-Duty 1: absent
Trip 1-Duty 1: absent
Trip 2-Duty 1: absent
Trip 3-Duty 1: absent
Trip 4-Duty 1: absent
Trip 5-Duty 1: absent
Trip 6-Duty 1: absent
Trip 7-Duty 1: absent
Trip 8-Duty 1: absent
Trip 9-Duty 1: absent
Duty_1: absent
Trip 0-Duty 2: absent
Trip 1-Duty 2: absent
Trip 2-Duty 2: absent
Trip 3-Duty 2: absent
Trip 4-Duty 2: absent
Trip 5-Duty 2: absent
Trip 6-Duty 2: absent
Trip 7-Duty 2: absent
Trip 8-Duty 2: absent
Trip 9-Duty 2: absent
Duty_2: absent


In [ ]:
msol = sub.solve()

report_solution(msol)

In [ ]:
sub.end_of(trips[i]) <= sub.end_of(trips[j]))

In [ ]:
start_of(trips[1]) <= 200

In [ ]:
less_or_equal(start_of(trips[0]), start_of(trips[1]))

In [ ]:
for i in range(len(trips)):
    for j in range(1, len(trips)):
        print(less_or_equal(trips[i].get_start() , trips[j].get_start()))


In [ ]:
for i in range(ntrips):
    sub.add(trip2trip[i] != 0)
    # sub.add(sum([trip2trip[j] == i for j in range(ntrips+1)]) == 1)

In [ ]:
for i in range(ntrips-1):
    for j in range(i + 1, ntrips):
        sub.add(end_before_start(trips[i], trips[j]))

In [ ]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [ ]:
trip_seq = sequence_var(trips)

In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")